In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
train_path  = "/content/drive/MyDrive/Breast Cancer/Final Work/Dataset/Grade_1"
train_names = os.listdir(train_path)
print(train_names)

['gamma_corr 0.png', 'gamma_corr 1.png', 'gamma_corr 2.png', 'gamma_corr 3.png', 'gamma_corr 4.png', 'gamma_corr 5.png', 'gamma_corr 6.png', 'gamma_corr 7.png', 'gamma_corr 8.png', 'gamma_corr 9.png', 'gamma_corr 10.png', 'gamma_corr 11.png', 'gamma_corr 12.png', 'gamma_corr 13.png', 'gamma_corr 14.png', 'gamma_corr 15.png', 'gamma_corr 16.png', 'gamma_corr 17.png', 'gamma_corr 18.png', 'gamma_corr 19.png', 'gamma_corr 20.png', 'gamma_corr 21.png', 'gamma_corr 22.png', 'gamma_corr 23.png', 'gamma_corr 24.png', 'gamma_corr 25.png', 'gamma_corr 26.png', 'gamma_corr 27.png', 'gamma_corr 28.png', 'gamma_corr 29.png', 'gamma_corr 30.png', 'gamma_corr 31.png', 'gamma_corr 32.png', 'gamma_corr 33.png', 'gamma_corr 34.png', 'gamma_corr 35.png', 'gamma_corr 36.png', 'gamma_corr 37.png', 'gamma_corr 38.png', 'gamma_corr 39.png', 'gamma_corr 40.png', 'gamma_corr 41.png', 'gamma_corr 42.png', 'gamma_corr 43.png', 'gamma_corr 44.png', 'gamma_corr 45.png', 'gamma_corr 46.png', 'gamma_corr 47.png', '

In [ ]:
import cv2
import numpy as np
import os
import glob
import time

tic = time.time()
coarseness_list=[]
contrast_list=[]
directionality_list=[]
roughness_list=[]

def coarseness(image, kmax):
    image = np.array(image)
    w = image.shape[0]
    h = image.shape[1]
    kmax = kmax if (np.power(2,kmax) < w) else int(np.log(w) / np.log(2))
    kmax = kmax if (np.power(2,kmax) < h) else int(np.log(h) / np.log(2))
    average_gray = np.zeros([kmax,w,h])
    horizon = np.zeros([kmax,w,h])
    vertical = np.zeros([kmax,w,h])
    Sbest = np.zeros([w,h])

    for k in range(kmax):
        window = np.power(2,k)
        for wi in range(w)[window:(w-window)]:
            for hi in range(h)[window:(h-window)]:
                average_gray[k][wi][hi] = np.sum(image[wi-window:wi+window, hi-window:hi+window])
        for wi in range(w)[window:(w-window-1)]:
            for hi in range(h)[window:(h-window-1)]:
                horizon[k][wi][hi] = average_gray[k][wi+window][hi] - average_gray[k][wi-window][hi]
                vertical[k][wi][hi] = average_gray[k][wi][hi+window] - average_gray[k][wi][hi-window]
        horizon[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))
        vertical[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))

    for wi in range(w):
        for hi in range(h):
            h_max = np.max(horizon[:,wi,hi])
            h_max_index = np.argmax(horizon[:,wi,hi])
            v_max = np.max(vertical[:,wi,hi])
            v_max_index = np.argmax(vertical[:,wi,hi])
            index = h_max_index if (h_max > v_max) else v_max_index
            Sbest[wi][hi] = np.power(2,index)

    fcrs = np.mean(Sbest)
    return fcrs


def contrast(image):
    image = np.array(image)
    image = np.reshape(image, (1, image.shape[0]*image.shape[1]))
    m4 = np.mean(np.power(image - np.mean(image),4))
    v = np.var(image)
    std = np.power(v, 0.5)
    alfa4 = m4 / np.power(v,2)
    fcon = std / np.power(alfa4, 0.25)
    return fcon

def directionality(image):
    image = np.array(image, dtype = 'int64')
    h = image.shape[0]
    w = image.shape[1]
    convH = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    convV = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    deltaH = np.zeros([h,w])
    deltaV = np.zeros([h,w])
    theta = np.zeros([h,w])

    # calc for deltaH
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaH[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convH))
    for wi in range(w)[1:w-1]:
        deltaH[0][wi] = image[0][wi+1] - image[0][wi]
        deltaH[h-1][wi] = image[h-1][wi+1] - image[h-1][wi]
    for hi in range(h):
        deltaH[hi][0] = image[hi][1] - image[hi][0]
        deltaH[hi][w-1] = image[hi][w-1] - image[hi][w-2]

    # calc for deltaV
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaV[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convV))
    for wi in range(w):
        deltaV[0][wi] = image[1][wi] - image[0][wi]
        deltaV[h-1][wi] = image[h-1][wi] - image[h-2][wi]
    for hi in range(h)[1:h-1]:
        deltaV[hi][0] = image[hi+1][0] - image[hi][0]
        deltaV[hi][w-1] = image[hi+1][w-1] - image[hi][w-1]

    deltaG = (np.absolute(deltaH) + np.absolute(deltaV)) / 2.0
    deltaG_vec = np.reshape(deltaG, (deltaG.shape[0] * deltaG.shape[1]))

    # calc the theta
    for hi in range(h):
        for wi in range(w):
            if (deltaH[hi][wi] == 0 and deltaV[hi][wi] == 0):
                theta[hi][wi] = 0;
            elif(deltaH[hi][wi] == 0):
                theta[hi][wi] = np.pi
            else:
                theta[hi][wi] = np.arctan(deltaV[hi][wi] / deltaH[hi][wi]) + np.pi / 2.0
    theta_vec = np.reshape(theta, (theta.shape[0] * theta.shape[1]))

    n = 16
    t = 12
    cnt = 0
    hd = np.zeros(n)
    dlen = deltaG_vec.shape[0]
    for ni in range(n):
        for k in range(dlen):
            if((deltaG_vec[k] >= t) and (theta_vec[k] >= (2*ni-1) * np.pi / (2 * n)) and (theta_vec[k] < (2*ni+1) * np.pi / (2 * n))):
                hd[ni] += 1
    hd = hd / np.mean(hd)
    hd_max_index = np.argmax(hd)
    fdir = 0
    for ni in range(n):
        fdir += np.power((ni - hd_max_index), 2) * hd[ni]
    return fdir

def roughness(fcrs, fcon):
    return fcrs + fcon

if __name__ == '__main__':
        
        # load the training dataset
        train_path  = "/content/drive/MyDrive/Breast Cancer/Final Work/Dataset/Grade_1"
        train_names = os.listdir(train_path)

        # loop over the training dataset
        cur_path = os.path.join(train_path, '*png')
        cur_label = train_names
        i = 0

        for file in glob.glob(cur_path):
            print('For image {} named {}:'.format(i+1,cur_label[i]))
            img = cv2.imread(file)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            print('Shape of image is: {} '.format(img.shape))
            fcrs = coarseness(img, 5)
            coarseness_list.append(fcrs)
            print("coarseness: %f" % fcrs);
            fcon = contrast(img)
            contrast_list.append(fcon)
            print("contrast: %f" % fcon)
            fdir= directionality(img)
            directionality_list.append(fdir)
            print("directionality: %f" % fdir)
            f_r=roughness(fcrs,fcon)
            roughness_list.append(f_r)
            print("roughness: %f" % f_r)
            print('\n\n')
            i+=1

toc = time.time()
#print("Computation time is {} minutes".format((toc-tic)/60)

For image 1 named gamma_corr 0.png:
Shape of image is: (1000, 1000) 
coarseness: 8.482649
contrast: 27.150020
directionality: 193.759360
roughness: 35.632669



For image 2 named gamma_corr 1.png:
Shape of image is: (1000, 1000) 
coarseness: 8.898144
contrast: 31.751899
directionality: 224.571741
roughness: 40.650043



For image 3 named gamma_corr 2.png:
Shape of image is: (1000, 1000) 
coarseness: 8.513999
contrast: 24.701777
directionality: 256.084677
roughness: 33.215776



For image 4 named gamma_corr 3.png:
Shape of image is: (1000, 1000) 
coarseness: 8.407529
contrast: 28.996641
directionality: 283.067050
roughness: 37.404170



For image 5 named gamma_corr 4.png:
Shape of image is: (1000, 1000) 
coarseness: 9.238046
contrast: 31.143715
directionality: 218.779135
roughness: 40.381761



For image 6 named gamma_corr 5.png:
Shape of image is: (1000, 1000) 
coarseness: 8.126395
contrast: 28.467641
directionality: 361.476063
roughness: 36.594036



For image 7 named gamma_corr 6.png

In [ ]:
print(coarseness_list)
print(len(coarseness_list))

In [ ]:
print(contrast_list)
print(len(contrast_list))

In [ ]:
print(directionality_list)
print(len(directionality_list))

In [ ]:
print(roughness_list)
print(len(roughness_list))

In [ ]:
# dictionary of lists 
import pandas as pd
dict = {'coarseness':coarseness_list , 'contrast': contrast_list, 'directionality': directionality_list,'roughness':roughness_list}   
df = pd.DataFrame(dict)

In [ ]:
df

In [ ]:
df.to_csv("/content/drive/MyDrive/Breast Cancer/Final Work/CSV File/tamura_grade_1.csv")

In [ ]:
import cv2
import numpy as np
import os
import glob
import time

tic = time.time()
coarseness_list=[]
contrast_list=[]
directionality_list=[]
roughness_list=[]

def coarseness(image, kmax):
    image = np.array(image)
    w = image.shape[0]
    h = image.shape[1]
    kmax = kmax if (np.power(2,kmax) < w) else int(np.log(w) / np.log(2))
    kmax = kmax if (np.power(2,kmax) < h) else int(np.log(h) / np.log(2))
    average_gray = np.zeros([kmax,w,h])
    horizon = np.zeros([kmax,w,h])
    vertical = np.zeros([kmax,w,h])
    Sbest = np.zeros([w,h])

    for k in range(kmax):
        window = np.power(2,k)
        for wi in range(w)[window:(w-window)]:
            for hi in range(h)[window:(h-window)]:
                average_gray[k][wi][hi] = np.sum(image[wi-window:wi+window, hi-window:hi+window])
        for wi in range(w)[window:(w-window-1)]:
            for hi in range(h)[window:(h-window-1)]:
                horizon[k][wi][hi] = average_gray[k][wi+window][hi] - average_gray[k][wi-window][hi]
                vertical[k][wi][hi] = average_gray[k][wi][hi+window] - average_gray[k][wi][hi-window]
        horizon[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))
        vertical[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))

    for wi in range(w):
        for hi in range(h):
            h_max = np.max(horizon[:,wi,hi])
            h_max_index = np.argmax(horizon[:,wi,hi])
            v_max = np.max(vertical[:,wi,hi])
            v_max_index = np.argmax(vertical[:,wi,hi])
            index = h_max_index if (h_max > v_max) else v_max_index
            Sbest[wi][hi] = np.power(2,index)

    fcrs = np.mean(Sbest)
    return fcrs


def contrast(image):
    image = np.array(image)
    image = np.reshape(image, (1, image.shape[0]*image.shape[1]))
    m4 = np.mean(np.power(image - np.mean(image),4))
    v = np.var(image)
    std = np.power(v, 0.5)
    alfa4 = m4 / np.power(v,2)
    fcon = std / np.power(alfa4, 0.25)
    return fcon

def directionality(image):
    image = np.array(image, dtype = 'int64')
    h = image.shape[0]
    w = image.shape[1]
    convH = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    convV = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    deltaH = np.zeros([h,w])
    deltaV = np.zeros([h,w])
    theta = np.zeros([h,w])

    # calc for deltaH
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaH[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convH))
    for wi in range(w)[1:w-1]:
        deltaH[0][wi] = image[0][wi+1] - image[0][wi]
        deltaH[h-1][wi] = image[h-1][wi+1] - image[h-1][wi]
    for hi in range(h):
        deltaH[hi][0] = image[hi][1] - image[hi][0]
        deltaH[hi][w-1] = image[hi][w-1] - image[hi][w-2]

    # calc for deltaV
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaV[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convV))
    for wi in range(w):
        deltaV[0][wi] = image[1][wi] - image[0][wi]
        deltaV[h-1][wi] = image[h-1][wi] - image[h-2][wi]
    for hi in range(h)[1:h-1]:
        deltaV[hi][0] = image[hi+1][0] - image[hi][0]
        deltaV[hi][w-1] = image[hi+1][w-1] - image[hi][w-1]

    deltaG = (np.absolute(deltaH) + np.absolute(deltaV)) / 2.0
    deltaG_vec = np.reshape(deltaG, (deltaG.shape[0] * deltaG.shape[1]))

    # calc the theta
    for hi in range(h):
        for wi in range(w):
            if (deltaH[hi][wi] == 0 and deltaV[hi][wi] == 0):
                theta[hi][wi] = 0;
            elif(deltaH[hi][wi] == 0):
                theta[hi][wi] = np.pi
            else:
                theta[hi][wi] = np.arctan(deltaV[hi][wi] / deltaH[hi][wi]) + np.pi / 2.0
    theta_vec = np.reshape(theta, (theta.shape[0] * theta.shape[1]))

    n = 16
    t = 12
    cnt = 0
    hd = np.zeros(n)
    dlen = deltaG_vec.shape[0]
    for ni in range(n):
        for k in range(dlen):
            if((deltaG_vec[k] >= t) and (theta_vec[k] >= (2*ni-1) * np.pi / (2 * n)) and (theta_vec[k] < (2*ni+1) * np.pi / (2 * n))):
                hd[ni] += 1
    hd = hd / np.mean(hd)
    hd_max_index = np.argmax(hd)
    fdir = 0
    for ni in range(n):
        fdir += np.power((ni - hd_max_index), 2) * hd[ni]
    return fdir

def roughness(fcrs, fcon):
    return fcrs + fcon

if __name__ == '__main__':
        
        # load the training dataset
        train_path  = "/content/drive/MyDrive/Breast Cancer/Final Work/Dataset/Grade_2"
        train_names = os.listdir(train_path)

        # loop over the training dataset
        cur_path = os.path.join(train_path, '*png')
        cur_label = train_names
        i = 0

        for file in glob.glob(cur_path):
            print('For image {} named {}:'.format(i+1,cur_label[i]))
            img = cv2.imread(file)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            print('Shape of image is: {} '.format(img.shape))
            fcrs = coarseness(img, 5)
            coarseness_list.append(fcrs)
            print("coarseness: %f" % fcrs);
            fcon = contrast(img)
            contrast_list.append(fcon)
            print("contrast: %f" % fcon)
            fdir= directionality(img)
            directionality_list.append(fdir)
            print("directionality: %f" % fdir)
            f_r=roughness(fcrs,fcon)
            roughness_list.append(f_r)
            print("roughness: %f" % f_r)
            print('\n\n')
            i+=1

toc = time.time()
#print("Computation time is {} minutes".format((toc-tic)/60)

In [ ]:
# dictionary of lists 
import pandas as pd
dict = {'coarseness':coarseness_list , 'contrast': contrast_list, 'directionality': directionality_list,'roughness':roughness_list}   
df = pd.DataFrame(dict)

In [ ]:
df.to_csv("/content/drive/MyDrive/Breast Cancer/Final Work/CSV File/tamura_grade_2.csv")

In [ ]:
import cv2
import numpy as np
import os
import glob
import time

tic = time.time()
coarseness_list=[]
contrast_list=[]
directionality_list=[]
roughness_list=[]

def coarseness(image, kmax):
    image = np.array(image)
    w = image.shape[0]
    h = image.shape[1]
    kmax = kmax if (np.power(2,kmax) < w) else int(np.log(w) / np.log(2))
    kmax = kmax if (np.power(2,kmax) < h) else int(np.log(h) / np.log(2))
    average_gray = np.zeros([kmax,w,h])
    horizon = np.zeros([kmax,w,h])
    vertical = np.zeros([kmax,w,h])
    Sbest = np.zeros([w,h])

    for k in range(kmax):
        window = np.power(2,k)
        for wi in range(w)[window:(w-window)]:
            for hi in range(h)[window:(h-window)]:
                average_gray[k][wi][hi] = np.sum(image[wi-window:wi+window, hi-window:hi+window])
        for wi in range(w)[window:(w-window-1)]:
            for hi in range(h)[window:(h-window-1)]:
                horizon[k][wi][hi] = average_gray[k][wi+window][hi] - average_gray[k][wi-window][hi]
                vertical[k][wi][hi] = average_gray[k][wi][hi+window] - average_gray[k][wi][hi-window]
        horizon[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))
        vertical[k] = horizon[k] * (1.0 / np.power(2, 2*(k+1)))

    for wi in range(w):
        for hi in range(h):
            h_max = np.max(horizon[:,wi,hi])
            h_max_index = np.argmax(horizon[:,wi,hi])
            v_max = np.max(vertical[:,wi,hi])
            v_max_index = np.argmax(vertical[:,wi,hi])
            index = h_max_index if (h_max > v_max) else v_max_index
            Sbest[wi][hi] = np.power(2,index)

    fcrs = np.mean(Sbest)
    return fcrs


def contrast(image):
    image = np.array(image)
    image = np.reshape(image, (1, image.shape[0]*image.shape[1]))
    m4 = np.mean(np.power(image - np.mean(image),4))
    v = np.var(image)
    std = np.power(v, 0.5)
    alfa4 = m4 / np.power(v,2)
    fcon = std / np.power(alfa4, 0.25)
    return fcon

def directionality(image):
    image = np.array(image, dtype = 'int64')
    h = image.shape[0]
    w = image.shape[1]
    convH = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    convV = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    deltaH = np.zeros([h,w])
    deltaV = np.zeros([h,w])
    theta = np.zeros([h,w])

    # calc for deltaH
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaH[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convH))
    for wi in range(w)[1:w-1]:
        deltaH[0][wi] = image[0][wi+1] - image[0][wi]
        deltaH[h-1][wi] = image[h-1][wi+1] - image[h-1][wi]
    for hi in range(h):
        deltaH[hi][0] = image[hi][1] - image[hi][0]
        deltaH[hi][w-1] = image[hi][w-1] - image[hi][w-2]

    # calc for deltaV
    for hi in range(h)[1:h-1]:
        for wi in range(w)[1:w-1]:
            deltaV[hi][wi] = np.sum(np.multiply(image[hi-1:hi+2, wi-1:wi+2], convV))
    for wi in range(w):
        deltaV[0][wi] = image[1][wi] - image[0][wi]
        deltaV[h-1][wi] = image[h-1][wi] - image[h-2][wi]
    for hi in range(h)[1:h-1]:
        deltaV[hi][0] = image[hi+1][0] - image[hi][0]
        deltaV[hi][w-1] = image[hi+1][w-1] - image[hi][w-1]

    deltaG = (np.absolute(deltaH) + np.absolute(deltaV)) / 2.0
    deltaG_vec = np.reshape(deltaG, (deltaG.shape[0] * deltaG.shape[1]))

    # calc the theta
    for hi in range(h):
        for wi in range(w):
            if (deltaH[hi][wi] == 0 and deltaV[hi][wi] == 0):
                theta[hi][wi] = 0;
            elif(deltaH[hi][wi] == 0):
                theta[hi][wi] = np.pi
            else:
                theta[hi][wi] = np.arctan(deltaV[hi][wi] / deltaH[hi][wi]) + np.pi / 2.0
    theta_vec = np.reshape(theta, (theta.shape[0] * theta.shape[1]))

    n = 16
    t = 12
    cnt = 0
    hd = np.zeros(n)
    dlen = deltaG_vec.shape[0]
    for ni in range(n):
        for k in range(dlen):
            if((deltaG_vec[k] >= t) and (theta_vec[k] >= (2*ni-1) * np.pi / (2 * n)) and (theta_vec[k] < (2*ni+1) * np.pi / (2 * n))):
                hd[ni] += 1
    hd = hd / np.mean(hd)
    hd_max_index = np.argmax(hd)
    fdir = 0
    for ni in range(n):
        fdir += np.power((ni - hd_max_index), 2) * hd[ni]
    return fdir

def roughness(fcrs, fcon):
    return fcrs + fcon

if __name__ == '__main__':
        
        # load the training dataset
        train_path  = "/content/drive/MyDrive/Breast Cancer/Final Work/Dataset/Grade_3"
        train_names = os.listdir(train_path)

        # loop over the training dataset
        cur_path = os.path.join(train_path, '*png')
        cur_label = train_names
        i = 0

        for file in glob.glob(cur_path):
            print('For image {} named {}:'.format(i+1,cur_label[i]))
            img = cv2.imread(file)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            print('Shape of image is: {} '.format(img.shape))
            fcrs = coarseness(img, 5)
            coarseness_list.append(fcrs)
            print("coarseness: %f" % fcrs);
            fcon = contrast(img)
            contrast_list.append(fcon)
            print("contrast: %f" % fcon)
            fdir= directionality(img)
            directionality_list.append(fdir)
            print("directionality: %f" % fdir)
            f_r=roughness(fcrs,fcon)
            roughness_list.append(f_r)
            print("roughness: %f" % f_r)
            print('\n\n')
            i+=1

toc = time.time()
#print("Computation time is {} minutes".format((toc-tic)/60)

In [ ]:
# dictionary of lists 
import pandas as pd
dict = {'coarseness':coarseness_list , 'contrast': contrast_list, 'directionality': directionality_list,'roughness':roughness_list}   
df = pd.DataFrame(dict)

In [ ]:
df.to_csv("/content/drive/MyDrive/Breast Cancer/Final Work/CSV File/tamura_grade_3.csv")